# Scikit-learn from CSV

This notebook reads the CSV data written out by the Dataflow program of [1_explore.ipynb](./1_explore.ipynb) and trains a scikit-learn model on Cloud ML Engine.


In [6]:
%pip install cloudml-hypertune

Note: you may need to restart the kernel to use updated packages.


In [1]:
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [14]:
%load_ext autoreload
%aimport ltgpred

## Train sklearn model locally

In [6]:
!mkdir -p preproc/csv
!gsutil cp gs://$BUCKET/lightning/preproc_0.02_32_2/csv/*-00000-* preproc/csv

mkdir: cannot create directory ‘preproc/csv’: File exists
Copying gs://cloud-training-demos-ml/lightning/preproc_0.02_32_2/csv/eval-00000-of-00253...
Copying gs://cloud-training-demos-ml/lightning/preproc_0.02_32_2/csv/train-00000-of-00522...
- [2 files][ 14.2 MiB/ 14.2 MiB]                                                
Operation completed over 2 objects/14.2 MiB.                                     


In [13]:
import pandas as pd
df = pd.read_csv('preproc/csv/train-00000-of-00522', 
        header=None,
        names=[
            'cx', 'cy', 'lat', 'lon', 'mean_ref_sm', 'max_ref_sm',
            'mean_ref_big', 'max_ref_big', 'ltg_sm', 'ltg_big', 'has_ltg'
        ])
del df['has_ltg']
#df = pd.get_dummies(df)
df.head()

,cx,cy,lat,lon,mean_ref_sm,max_ref_sm,mean_ref_big,max_ref_big,ltg_sm,ltg_big
0,1186,770,47.72,-109.60,0.397961,0.454567,0.489967,0.885870,0.00,0.017041
1,586,898,35.72,-107.04,0.391342,0.500481,0.407276,0.885218,0.00,0.044970
2,626,1010,36.52,-104.80,0.574724,0.818150,0.625667,0.931585,0.92,0.288521
3,42,1090,24.84,-103.20,0.688098,0.831441,0.527593,0.894942,0.16,0.242840
4,318,1298,30.36,-99.04,0.481775,0.854034,0.469902,0.869434,0.00,0.084734


In [ ]:
%%bash
export CLOUDSDK_PYTHON=$(which python3)
OUTDIR=skl_trained
DATADIR=${PWD}/preproc/csv
rm -rf $OUTDIR
gcloud ml-engine local train \
    --module-name=trainer.train_skl --package-path=${PWD}/ltgpred/trainer \
    -- \
    --job-dir=$OUTDIR --train_data=${DATADIR}/train* --eval_data=${DATADIR}/eval*

## Training sklearn model on CMLE

In [18]:
%%writefile largemachine.yaml
trainingInput:
  scaleTier: CUSTOM
  masterType: complex_model_l

Overwriting largemachine.yaml


In [19]:
%%bash
export CLOUDSDK_PYTHON=$(which python3)
OUTDIR=gs://${BUCKET}/lightning/skl_trained
DATADIR=gs://$BUCKET/lightning/preproc_0.02_32_2/csv
JOBNAME=ltgpred_skl_$(date -u +%y%m%d_%H%M%S)
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
    --module-name=ltgpred.trainer.train_skl --package-path=${PWD}/ltgpred --job-dir=$OUTDIR \
    --region=${REGION} --scale-tier=custom --config=largemachine.yaml \
    --python-version=3.5 --runtime-version=1.9 \
    -- \
    --train_data=${DATADIR}/train-001* --eval_data=${DATADIR}/eval-0000*

jobId: ltgpred_skl_190423_034420
state: QUEUED


ERROR: Python 3 and later is not compatible with the Google Cloud SDK. Please use Python version 2.7.x.

If you have a compatible Python interpreter installed, you can use it by setting
the CLOUDSDK_PYTHON environment variable to point to it.

Job [ltgpred_skl_190423_034420] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe ltgpred_skl_190423_034420

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs ltgpred_skl_190423_034420


When I ran it, training finished with a RMSE=0.43.
This can serve as a benchmark.

Note, however, that I trained and evaluated on a subset of the data, since even the "largemachine" doesn't have the memory needed to hold entire dataset

Copyright 2018 Google Inc. Licensed under the Apache License, Version 2.0 (the \"License\"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License